In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [3]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-a2cba1cx
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-a2cba1cx
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=baf8c3d129387251b8550be8160307b4cb66f454d80c0740634cdda7ca210027
  Stored in directory: /tmp/pip-ephem-wheel-cache-xxq16hdw/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [4]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [5]:
!nvidia-smi

Fri Apr 10 06:14:03 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [6]:
!ls

sample_data  src


In [33]:
%%cuda --name helloCUDA.cu
#include <iostream>
#include <random>
#include <algorithm>

#define LOOP(x) for(int t##x = 0; t##x < x; t##x++)

using namespace std;

void gpu_error(cudaError_t const &code) 
{
    if(code != cudaSuccess)
    {
        cerr << "GPUError: Code " << code << " : " << cudaGetErrorString(code) << endl;
        exit( EXIT_FAILURE );
    }
}

__global__ void tile(float *devin, float *devout, float *devsum, int h, int w)
{
    float thrtile[4][4];
    
    int /*bs,*/ p, q, ch;
    // bs = gridDim.x;
    p = gridDim.y;
    q = gridDim.z;
    ch = blockDim.x;
    
    int tbs, tp, tq, tch;
    tbs = blockIdx.x;
    tp = blockIdx.y;
    tq = blockIdx.z;
    tch = threadIdx.x;

    // copy the tiles to thrtile

    int offset1 = (tbs*ch + tch)*h*w;

    // float *t = thrtile;
 
    for(int th = 2*tp, i = 0; i < 4; th++, i++)
    {
        for(int tw = 2*tq, j = 0; j < 4; tw++, j++)
        {
            thrtile[i][j] = devin[offset1 + th*w + tw];
        }
    }

    // copy thrtile to devout for testing

    int offset2 = (((tbs*p + tp)*q + tq)*ch + tch)*16;

    for(int i = 0; i < 4; i++)
    {
        for(int j = 0; j < 4; j++)
        {
            devout[offset2 + i*4 + j] = thrtile[i][j];
        }
    }

    // sum along the channels, using log n summing

    // int k = ch, j = tch;

    int offset3 = ((tbs*p + tp)*q + tq)*ch*16;

    for(int s = 1; s < ch; s *= 2)
    {
        if(tch % (2*s) == 0 && tch+s < ch)
        {
            for(int i = 0; i < 4; i++)
            {
                for(int j = 0; j < 4; j++)
                {
                    devout[offset3 + tch*16 + i*4 + j] += devout[offset3 + (tch+s)*16 + i*4 + j];
                }
            }
        }
        __syncthreads();
    }

    if(tch/*%ch*/ == 0) // can do with tch == 0
    {
        int offset = ((tbs*p + tp)*q + tq)*16;
        for(int i = 0; i < 4; i++)
        {
            for(int j = 0; j < 4; j++)
            {
                devsum[offset + i*4 + j] = devout[offset3 + /*tch*16*/ +i*4 + j];
            }
        }
    }

}

__global__ void paddev(float *devin, float *devinnopad, int h, int w, int pad)
{
    int newh = gridDim.y;
    int neww = gridDim.z;
    int tbsch = blockIdx.x;
    int tnewh = blockIdx.y;
    int tneww = blockIdx.z;
    int newhw = newh*neww;
    int hw = h*w;
    int th = tnewh-pad;
    int tw = tneww-pad;
    
    if(th >= 0 && th < h && tw >= 0 && tw < w)
    {
        devin[tbsch*newhw + tnewh*neww + tneww] = devinnopad[tbsch*hw + th*w + tw];
    }
    else
    {
        devin[tbsch*newhw + tnewh*neww + tneww] = 0;
    }
    
}

void tilehost(int och, int ch, int bs, int &h, int &w, float *&in, int &p, int &q, int &outsize, float *&out, int &sumsize, float *&sum, int pad, float *&padded)
{
    float *devin, *devinnopad;
    int insize = bs * ch * h * w * sizeof(float);
    gpu_error(cudaMalloc((void **) & devinnopad, insize));
    gpu_error(cudaMemcpy(devinnopad, in, insize, cudaMemcpyHostToDevice));

    int newh, neww;
    newh = h + 2*pad;
    neww = w + 2*pad;
    if(newh%2)
        newh++;
    if(neww%2)
        neww++;
    if(newh < 4)
        newh = 4;
    if(neww < 4)
        neww = 4;

    insize = bs * ch * newh * neww * sizeof(float);
    gpu_error(cudaMalloc((void **) & devin, insize));

    // call padding
    dim3 padgrid(bs*ch, newh, neww);
    dim3 padblock(1, 1, 1);
 
    paddev<<<padgrid,padblock>>>(devin, devinnopad, h, w, pad);

    gpu_error(cudaFree(devinnopad));
 
    padded = new float[insize/sizeof(float)];
    
    gpu_error(cudaMemcpy(padded, devin, insize, cudaMemcpyDeviceToHost));
    
    h = newh;
    w = neww;

    
    // int p, q;
    p = max((h-2)/2, 0);
    q = max((w-2)/2, 0);
    
    float *devout, *devsum;
    devout = devsum = nullptr;
    outsize = bs * p * q * ch * 4 * 4 * sizeof(float);
    sumsize = bs * p * q * 4 * 4 * sizeof(float);

    gpu_error(cudaMalloc((void **) & devout, outsize));
    gpu_error(cudaMalloc((void **) & devsum, sumsize));
    
    // call the kernel function for tiling
    
    dim3 grid(bs, p, q);  // 3-D
    dim3 block(ch, 1, 1); // 1-D

    tile<<<grid, block>>>(devin, devout, devsum, h, w);

    // copy from device to host to out.

    delete in;
    out = new float[outsize/sizeof(float)];
    sum = new float[sumsize/sizeof(float)];

    gpu_error(cudaMemcpy(out, devout, outsize, cudaMemcpyDeviceToHost));
    gpu_error(cudaMemcpy(sum, devsum, sumsize, cudaMemcpyDeviceToHost));

    gpu_error(cudaFree(devin));
    gpu_error(cudaFree(devout));
    gpu_error(cudaFree(devsum));
    
}

void padding(float *&in, int bs, int ch, int &h, int &w, int pad)
{
    // Here, after adding pad we also round up h, w to become a multiple of tile.
    // This is done such that the actual matrix is present at top left of this matrix.

    int newh, neww;
    newh = h + 2*pad;
    neww = w + 2*pad;
    if(newh%2)
        newh++;
    if(neww%2)
        neww++;
    if(newh < 4)
        newh = 4;
    if(neww < 4)
        neww = 4;

    int slices = bs*ch;
    int newhw = newh*neww;
    float *newin = new float[slices*newhw];
    float *tin = in, *tnewin = newin;
    LOOP(slices)
    {
        LOOP(newh)
        {
            LOOP(neww)
            {
                if(tnewh >= pad && tnewh-pad < h && tneww >= pad && tneww-pad < w)
                {
                    *(tnewin++) = *(tin++);
                }
                else
                {
                    *(tnewin++) = 0;
                }
            }
        }
    }

    delete in;
    in = newin;

    h = newh;
    w = neww;

}

int main(void) 
{
    auto engine = default_random_engine(time(nullptr));
    auto rng = uniform_real_distribution<float>();

    int bs, ch, h, w, p, q, oldh, oldw, pad;
    
    bs = 3;
    ch = 2;
    oldh = h = 3;
    oldw = w = 3;
    pad = 1;
    
    int insize = bs * ch * h * w * sizeof(float);
    int outsize, sumsize;
 
    float *in = new float[insize/sizeof(float)];
    float *t = in;
    float *out, *sum, *padded;
 
    LOOP(bs)
    {
        LOOP(ch)
        {
            LOOP(h)
            {
                LOOP(w)
                {
                    *(t++) = rng(engine);
                }
            }
        }
    }
 
    LOOP(bs)
    {
        cout<<"{ ";
        LOOP(ch)
        {
            cout<<"{ ";
            LOOP(h)
            {
                cout<<"{ ";
                LOOP(w)
                {
                    cout<<in[((tbs*ch+tch)*h+th)*w+tw]<<" ";
                }
                cout<<"}\n";
            }
            cout<<"}\n";
        }
        cout<<"}\n";
    }

    //cout<<"\nPadding\n";

    //padding(in, bs, ch, h, w, pad);

   // cout<<"\nPadding done\n";
    
    /*
    LOOP(bs)
    {
        cout<<"{ ";
        LOOP(ch)
        {
            cout<<"{ ";
            LOOP(h)
            {
                cout<<"{ ";
                LOOP(w)
                {
                    cout<<in[((tbs*ch+tch)*h+th)*w+tw]<<" ";
                }
                cout<<"}\n";
            }
            cout<<"}\n";
        }
        cout<<"}\n";
    }
    */

    cout<<"\nPadding and Tiling and Summing\n";

    tilehost(1, ch, bs, h, w, in, p, q, outsize, out, sumsize, sum, pad, padded);
 
    cout<<"\nPadding finished\n\n";
 
    LOOP(bs)
    {
        cout<<"{ ";
        LOOP(ch)
        {
            cout<<"{ ";
            LOOP(h)
            {
                cout<<"{ ";
                LOOP(w)
                {
                    cout<<padded[((tbs*ch+tch)*h+th)*w+tw]<<" ";
                }
                cout<<"}\n";
            }
            cout<<"}\n";
        }
        cout<<"}\n";
    }
    
    cout<<"\nTiling finished\n\n";

    /*
    
    LOOP(bs)
    {
        cout<<"{ ";
        LOOP(p)
        {
            cout<<"{ ";
            LOOP(q)
            {
                cout<<"{ ";
                LOOP(ch)
                {
                    cout<<"{ ";
                    for(int i = 0; i < 4; i++)
                    {
                        for(int j = 0; j < 4; j++)
                        {
                            cout<<out[((((tbs*p+tp)*q+tq)*ch+tch)*4+i)*4+j]<<",";
                        }
                        cout<<";\n";
                    }
                    cout<<"}\n";
                }
                cout<<"}\n";
            }
            cout<<"}\n";
        }
        cout<<"}\n";
    }
 
    */

    cout<<"\nSumming finished\n\n";

    LOOP(bs)
    {
        cout<<"{ ";
        LOOP(p)
        {
            cout<<"{ ";
            LOOP(q)
            {
                cout<<"{ ";
                for(int i = 0; i < 4; i++)
                {
                    for(int j = 0; j < 4; j++)
                    {
                        cout<<sum[(((tbs*p+tp)*q+tq)*4+i)*4+j]<<",";
                    }
                    cout<<";\n";
                }
                cout<<"}\n";
            }
            cout<<"}\n";
        }
        cout<<"}\n";
    }

    return 0;
}

'File written in /content/src/helloCUDA.cu'

In [34]:
!nvcc -std=c++11 /content/src/helloCUDA.cu -o /content/src/helloCUDA

/content/src/helloCUDA.cu(242): warning: variable "oldh" was set but never used

/content/src/helloCUDA.cu(242): warning: variable "oldw" was set but never used



In [35]:
!/content/src/helloCUDA #< input.txt

{ { { 0.539839 0.0733177 0.249826 }
{ 0.832637 0.125438 0.243395 }
{ 0.732837 0.785007 0.609485 }
}
{ { 0.613576 0.378992 0.712288 }
{ 0.425089 0.477968 0.213586 }
{ 0.734359 0.374151 0.350825 }
}
}
{ { { 0.308657 0.602229 0.668334 }
{ 0.694463 0.837411 0.365047 }
{ 0.338661 0.874876 0.0443979 }
}
{ { 0.195269 0.88367 0.848348 }
{ 0.177174 0.767741 0.418122 }
{ 0.374486 0.987078 0.825242 }
}
}
{ { { 0.842272 0.0646405 0.413692 }
{ 0.922955 0.109949 0.91089 }
{ 0.333096 0.349724 0.818398 }
}
{ { 0.817012 0.513934 0.697436 }
{ 0.805758 0.368755 0.657222 }
{ 0.928257 0.207121 0.0817278 }
}
}

Padding and Tiling and Summing

Padding finished

{ { { 0 0 0 0 0 0 }
{ 0 0.539839 0.0733177 0.249826 0 0 }
{ 0 0.832637 0.125438 0.243395 0 0 }
{ 0 0.732837 0.785007 0.609485 0 0 }
{ 0 0 0 0 0 0 }
{ 0 0 0 0 0 0 }
}
{ { 0 0 0 0 0 0 }
{ 0 0.613576 0.378992 0.712288 0 0 }
{ 0 0.425089 0.477968 0.213586 0 0 }
{ 0 0.734359 0.374151 0.350825 0 0 }
{ 0 0 0 0 0 0 }
{ 0 0 0 0 0 0 }
}
}
{ { { 0 0 0 0 0 0 }
{ 